In [ ]:
!pip install google-cloud --user
!pip install --upgrade google-cloud-bigquery[pandas] --user
!pip install google-cloud-storage --user

In [ ]:
%load_ext google.cloud.bigquery

### Setup variables

Change ***GCP_PROJECT_ID*** with your project_id

In [ ]:
SERVICE_ACCOUNT='jupytersa'
JSON_FILE_NAME = '/home/jovyan/work/sa1.json'
GCP_PROJECT_ID='<GCP_PROJECT_ID>'

### Create Service Account

In [ ]:
import subprocess
import sys
import logging

logger = logging.Logger('catch_all')

def run_command(parameters):
    try:
        # """Prints and runs a command."""
        return subprocess.check_output(parameters)
    except BaseException as e: 
       logger.error(e) 
       logger.error('ERROR: Looking in jupyter console for more information')

In [ ]:
run_command([
        'gcloud', 'iam', 'service-accounts',
        'create', SERVICE_ACCOUNT,
        '--display-name', "Service Account for BETA SCC API",
        '--project', GCP_PROJECT_ID
])

In [ ]:
IAM_ROLES = [
    'roles/editor'
]

for role in IAM_ROLES:
    run_command([
        'gcloud', 'projects', 'add-iam-policy-binding',GCP_PROJECT_ID,
        '--member', 'serviceAccount:{}@{}.iam.gserviceaccount.com'.format(SERVICE_ACCOUNT, GCP_PROJECT_ID),
        '--quiet',  '--role', role
    ])

In [ ]:
run_command([
        'gcloud', 'iam', 'service-accounts',
        'keys', 'create',JSON_FILE_NAME ,
        '--iam-account', 
        '{}@{}.iam.gserviceaccount.com'.format(SERVICE_ACCOUNT, GCP_PROJECT_ID)
])

In [ ]:
import logging
import json
import os
from datetime import datetime
import pprint

from googleapiclient import discovery
from oauth2client.service_account import ServiceAccountCredentials

# Default scope to get access token
_SCOPE = 'https://www.googleapis.com/auth/cloud-platform'
CLIENT_DISCOVERY_SERVICE_URL_BASE = 'https://securitycenter.googleapis.com/$discovery/rest'

#### GCP API (https://github.com/googleapis/google-cloud-python)

### Storage

In [ ]:
from google.cloud import storage

# Explicitly use service account credentials by specifying the private key
    # file.
storage_client = storage.Client.from_service_account_json(JSON_FILE_NAME)

# Make an authenticated API request
buckets = list(storage_client.list_buckets())
print(buckets)

### BigQuery

In [ ]:
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json(JSON_FILE_NAME)
# Perform a query.
QUERY = (
    'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.name)